In [ ]:
import pandas as pd
import os

neues_verzeichnis = "C:\\Users\\Lucian\\Desktop\\WebMining_final"
os.chdir(neues_verzeichnis)

# Datei einlesen
file_path = "test_steam_game_genres.csv"  # Falls nötig, Dateipfad anpassen
df_genres = pd.read_csv(file_path)
# Genres in einzelne Einträge aufsplitten und Bereinigung durchführen
df_genres['genres'] = df_genres['genres'].astype(str).fillna('')  # Sicherstellen, dass alle Werte Strings sind
all_genres = df_genres['genres'].str.split(',').explode().str.strip()  # Splitten und bereinigen

# Alle einzigartigen Genres & Varianten extrahieren
unique_genres = all_genres.unique()

# Ergebnisse anzeigen
print(f"Anzahl einzigartiger Genres: {len(unique_genres)}")
print("Einzigartige Genre-Bezeichnungen:")
for genre in sorted(unique_genres):
    print(genre)

# Einlesen des Outputs in ChatGPT, um für nächsten Schritt Genre Mapping zu erstellen. Codelösung für Erstellung des Genre Mappings in Version 2.0

Anzahl einzigartiger Genres: 67
Einzigartige Genre-Bezeichnungen:
Abenteuer
Accounting
Acesso Antecipado
Action
Adventure
Akcja
Aksiyon
Animation & Modeling
Audio Production
Aventura
Ação
Bağımsız Yapımcı
Casual
Design & Illustration
Early Access
Education
Estratégia
Free To Play
Game Development
Gore
Indie
Kostenlos spielbar
MMO
Macera
Massively Multiplayer
Multijogador Massivo
Niezależne
Nudity
Photo Editing
RPG
Racing
Rekreacyjne
Rennspiele
Rollenspiel
Sexual Content
Simulation
Simulationen
Software Training
Sport
Sports
Strategy
Symulacje
Tasarım ve İllüstrasyon
Utilities
Video Production
Violent
Wczesny dostęp
Web Publishing
Wyścigowe
Yardımcı Uygulamalar
nan
Інді
Анімація та моделювання
Бесплатные
Бойовики
Вільний доступ
Дизайн та ілюстрація
Инди
Казуальні ігри
Пригоди
Рольові ігри
Симуляторы
Службові програми
Створення відео
Створення звуку
Стратегії
Экшены


In [36]:
# Normalisierungstabelle für Genres (Übersetzungen & Synonyme)
genre_mapping = {
    "Abenteuer": "Adventure", "Aventura": "Adventure", "Macera": "Adventure", "Пригоди": "Adventure",
    "Action": "Action", "Akcja": "Action", "Aksiyon": "Action", "Экшены": "Action", "Бойовики": "Action", "Ação": "Action",
    "Casual": "Casual", "Kazualni": "Casual", "Casualni": "Casual", "Казуальні ігри": "Casual",
    "Early Access": "Early Access", "Acesso Antecipado": "Early Access", "Wczesny dostęp": "Early Access",
    "Free To Play": "Free-to-Play", "Kostenlos spielbar": "Free-to-Play", "Бесплатные": "Free-to-Play", "Вільний доступ": "Free-to-Play",
    "Indie": "Indie", "Independente": "Indie", "Bağımsız Yapımcı": "Indie", "Niezależne": "Indie",
    "Инди": "Indie", "Інді": "Indie", "MMO": "MMO", "Massively Multiplayer": "MMO", "Multijogador Massivo": "MMO",
    "RPG": "RPG", "Rollenspiel": "RPG", "Рольові ігри": "RPG", "Simulation": "Simulation",
    "Simulationen": "Simulation", "Symulacje": "Simulation", "Симуляторы": "Simulation",
    "Sports": "Sports", "Sport": "Sports", "Rekreacyjne": "Sports", "Wyścigowe": "Sports",
    "Strategy": "Strategy", "Estratégia": "Strategy", "Стратегії": "Strategy",
    "Racing": "Racing", "Rennspiele": "Racing", "Wyścigowe": "Racing",
    "Utilities": "Utilities", "Yardımcı Uygulamalar": "Utilities", "Службові програми": "Utilities",
    "Violent": "Violent", "Gore": "Violent", "Sexual Content": "Adult Content", "Nudity": "Adult Content",
    "Animation & Modeling": "Design & Illustration", "Tasarım ve İllüstrasyon": "Design & Illustration", "Дизайн та ілюстрація": "Design & Illustration",
    "Video Production": "Media & Production", "Створення відео": "Media & Production",
    "Audio Production": "Media & Production", "Створення звуку": "Media & Production",
    "Web Publishing": "Software & Tools", "Software Training": "Software & Tools", "Game Development": "Software & Tools",
    "Accounting": "Software & Tools", "Education": "Software & Tools"
}

normalized_genres = df_genres.copy()

# Genres splitten, normalisieren und wieder zusammenfügen
normalized_genres['genres'] = normalized_genres['genres'].str.split(',')
normalized_genres = normalized_genres.explode('genres')
normalized_genres['genres'] = normalized_genres['genres'].str.strip()  
normalized_genres['genres'] = normalized_genres['genres'].fillna('Unknown')

# Normalisierung durchführen
normalized_genres['genres'] = normalized_genres['genres'].replace(genre_mapping)

# Nach Normalisierung zusammenfügen, um Duplikate zu vermeiden
normalized_genres["genres"] = normalized_genres.groupby('appid')['genres'].transform(lambda x: ', '.join(sorted(set(x))))

normalized_genres = normalized_genres.drop_duplicates(subset=['appid', 'genres'])

# Prüfen, ob alle Genres ersetzt wurden
clean_genres = normalized_genres['genres'].str.split(',').explode().str.strip().unique()

print(f"Anzahl einzigartiger Genres nach Normalisierung: {len(clean_genres)}")
print("Normalisierte Genre-Bezeichnungen:")
for genre in sorted(clean_genres):
    print(genre)

Anzahl einzigartiger Genres nach Normalisierung: 21
Normalisierte Genre-Bezeichnungen:
Action
Adult Content
Adventure
Casual
Design & Illustration
Early Access
Free-to-Play
Indie
MMO
Media & Production
Photo Editing
RPG
Racing
Simulation
Software & Tools
Sports
Strategy
Unknown
Utilities
Violent
Анімація та моделювання


In [37]:
normalized_genres

,appid,genres
0,570,"Action, Free-to-Play, Strategy"
1,730,"Action, Free-to-Play"
2,786520,"Early Access, Utilities"
3,1056640,"Action, Free-to-Play, MMO, RPG"
4,8500,"Action, Free-to-Play, MMO, RPG, Strategy"
...,...,...
19523,361160,Unknown
19524,1151130,Unknown
19525,1833490,Unknown
19526,1073900,Unknown
